In [4]:
import sys
import pandas as pd
import os
sys.path.insert(0,r'../../pydaisy/')
import matplotlib.pyplot as plt
import numpy as np 
import datetime as datetime

from pydaisy.Daisy import *

In [36]:
# Import excel w scenario information
# the crop rotations
rota = pd.read_excel('../common/masterinput_v1.xlsx', sheet_name= 'Rotations')
# import the crops: daisynames, plowing, sowing and harvest dates
crops = pd.read_excel('../common/masterinput_v1.xlsx',sheet_name= 'Crops')
crops.index=crops['Crop']
# import the manure IDs, amounts, and types
manure = pd.read_excel('../common/masterinput_v1.xlsx', sheet_name = 'Manure')
# import table with file names for weather data, soil data, initialization file, initial SOC content
conditions = pd.read_excel('../common/masterinput_v1.xlsx', sheet_name = 'soil_climate_more')
# rota.head
rota.describe()

,ID,KK2,KK3,K4,K5,K6,K7,K8,K9,K10,...,P5,PK8,P9,PK10,PØ11,PØ12,SK1,S1,S4,SØ6
count,11,9,8,9,9,9,9,9,7,8,...,8,7,9,8,8,8,8,9,9,7
unique,11,3,6,7,9,8,7,6,4,4,...,6,2,8,7,7,7,4,4,8,6
top,Konv,Silomajs,Kløvergræs,Kløvergræs,Kløvergræs,Kløvergræs,Kløvergræs,Kløvergræs,GræsP,GræsV,...,x,Vinterhvede,Frøgræs,Sukkerroer,Vårhvede GG,Vårhvede GG,Vårbyg,Vårbyg E,Vinterhvede,Kløvergræs GG
freq,1,6,2,3,1,2,3,4,4,5,...,2,6,2,2,2,2,5,6,2,2


In [39]:

conditions.head()



,Jordtyper,jb1,jb4,jb6
0,Kulstofsindhold,3,NaN,NaN
1,Initialisering,1,2,3.0
2,Klima,weather.dwf,weather2.dwf,NaN


In [38]:
"""
Gør noget med vinterbyg PE hvor sowing2 ligger året efter sowing 1 men en måned tidligere
"""
    
path=r'../Run'

# if __name__ =='__main__':

template = DaisyModel(os.path.join(path, 'Scenarier_v1.dai'))
i=0

for c in range(0, 1):
    newfile= template.copy()    
    block = newfile.Input['defaction'][1]
    
    soiltype = conditions.columns[c]
        for soil in range(0,3):
            block.Children.append(DaisyEntry())
    for i in range(1, 17):
        columnname=rota.columns[i]

        for year in range(0,6):
            if not pd.isna(rota[columnname][year]):
                cropname = rota[columnname][year].strip()
                if not pd.isna(crops['Plowing'][cropname]):
                    block.Children.append(DaisyEntry('wait_mm_dd', [crops['Plowing'][cropname].strftime('%m %d')]))  
                    block.Children.append(DaisyEntry('plowing',[]))
                if not pd.isna(crops['Sowing1'][cropname]):
                    block.Children.append(DaisyEntry('wait_mm_dd', [crops['Sowing1'][cropname].strftime('%m %d')]))  
                    for c in crops['Daisynavn1'][cropname].split(','):
                        block.Children.append(DaisyEntry('sow', ['"' + str(c.strip()) +'"']))
                if not pd.isna(crops['Sowing2'][cropname]):
                    block.Children.append(DaisyEntry('wait_mm_dd', [crops['Sowing2'][cropname].strftime('%m %d')]))  
                    for c in crops['Daisynavn2'][cropname].split(','):
                        block.Children.append(DaisyEntry('sow', ['"' + str(c.strip()) +'"']))
                if not pd.isna(crops['Harvest1'][cropname]):
                    harvestdates=[]
                    if isinstance(crops['Harvest1'][cropname],datetime): 
                        harvestdates.append(crops['Harvest1'][cropname])
                    else:
                        for s in crops['Harvest1'][cropname].split(','):
                            harvestdates.append(datetime(2010, int(s.split('/')[1]), int(s.split('/')[0])))
                    for date in harvestdates:
                        block.Children.append(DaisyEntry('wait_mm_dd', [date.strftime('%m %d')]))
                        for c in crops['Daisynavn1'][cropname].split(','):
                            block.Children.append(DaisyEntry('harvest', ['"' + str(c.strip()) +'"']))
                if not pd.isna(crops['Harvest2'][cropname]):
                    harvestdates=[]
                    if isinstance(crops['Harvest2'][cropname],datetime): 
                        harvestdates.append(crops['Harvest2'][cropname])
                    else:
                        for s in crops['Harvest2'][cropname].split(','):
                            harvestdates.append(datetime(2010, int(s.split('/')[1]), int(s.split('/')[0])))
                    for date in harvestdates:
                        block.Children.append(DaisyEntry('wait_mm_dd', [date.strftime('%m %d')]))
                        for c in crops['Daisynavn2'][cropname].split(','):
                            block.Children.append(DaisyEntry('harvest', ['"' + str(c.strip()) +'"']))
    newfile.save_as(os.path.join(path, soiltype, columnname,'model.dai'))
        
    
    
    
    
    

In [13]:
ip

TypeError: cannot do label indexing on <class 'pandas.core.indexes.base.Index'> with these indexers [nan] of <class 'float'>

In [17]:
pd.isna(rota[columnname][year])






True

In [ ]:

for i in range(0,len()):
    block.Children.append(DaisyEntry('wait_mm_dd', [str(df['date'][i].month), str(df['date'][i].day)]))
        
    if rota[2:,1:][i]=='sow':
        for crop in df['what'][i].split(','):
                sow = DaisyEntry('sow', ['"' + crop.strip() +'"'])
                block.Children.append(sow)  

  